## Project Description
@todo


### Nomenclature
Property names are derived out of entities they join.

The ground/root entity is called ent0
Add the word out/in when required as shown in photo here: 
<img src="files/resources/nomenclature.png">
@todo


In [1]:
#Importing some external libraries
from pprint import pprint
import networkx as nx
import pickle
import json
import copy

#Importing internal classes/libraries
import utils.dbpedia_interface as db_interface
import utils.natural_language_utilities as nlutils
import utils.subgraph as subgraph

#@TODO: put this class there


In [2]:
'''
    Initializing some stuff. Namely: DBpedia interface class.
    Reading the list of 'relevant' properties.
'''

dbp = None	#DBpedia interface object #To be instantiated when the code is run by main script/unit testing script
relevant_properties = open('resources/relation_whitelist.txt').read().split('\n')    #Contains the whitelisted props types

In [3]:
'''
    Some SPARQL Queries.
    Since this part of the code requires sending numerous convoluted queries to DBpedia, 
        we best not clutter the DBpedia interface class and rather simply declare them here.
        
    Note: The names here can be confusing. Refer to the diagram above to know what each SPARQL query tries to do.
'''

one_triple_right = '''
			SELECT DISTINCT ?p ?e 
			WHERE { 
				<%(e)s> ?p ?e 
			}'''

one_triple_left = '''
			SELECT DISTINCT ?e ?p
			WHERE {
				?e ?p <%(e)s>
			}'''


#@TODO: TESTING

In [4]:
'''
    This cell houses the script which will build a subgraph as shown in picture above for each a given URI.
    @TODO: take care of literals
    @TODO: do something in cases where certain nodes of the local subgraph are not found. 
            Will the code throw errors? How to you take care of them
'''

def insert_triple_in_subgraph(G, _results, _labels, _direction, _origin_node, _filter_properties = True, _filter_literals = True):
    '''
        Function used to push the results of different queries into the subgraph.
        USAGE: only within the get_local_subgraph function.
        
        INPUTS:
        _subgraph: the subgraph object within which the triples are to be pushed
        _results: a result list which contains the sparql variables 'e' and 'p'. 
                They can be of either left or right queries as the cell above
        _labels: a tuple with three strings, which depict the nomenclature of the resources to be pushed
        _direction: True -> one triple right; False -> one triple left
        _origin_node: the results variable only gives us one p and one e. 
                Depending on the direction, this node will act as the other e to complete the triple  
        _filter_properties: if True, only properties existing in properties whitelist will be pushed in.
    '''
    
    
    for result in _results[u'results'][u'bindings']:
        #Parse the results into local variables (for readibility)
        prop = result[u'p'][u'value']
        ent = result[u'e'][u'value']
        
        if _filter_literals:
        	if nlutils.has_literal(ent):
        		continue

        if _filter_properties:
            
            #Filter results based on important properties
            if not prop.split('/')[-1] in relevant_properties:
                continue
        
        #Finally, insert, based on direction
        if _direction == True:
            #Right
            subgraph.insert(G=G, data=[ (_labels[0],_origin_node), (_labels[1],prop), (_labels[2],ent) ])
            
        elif _direction == False:
            #Left
            subgraph.insert(G=G, data=[(_labels[0],ent), (_labels[1],prop), (_labels[2],_origin_node) ])       
        
def get_local_subgraph(_uri):
    #Collecting required variables: DBpedia interface, and a new subgraph
    global dbp
    
    #Create a new graph
    G = nx.DiGraph()
    access = subgraph.accessGraph(G)
    
    
    ########### e ?p ?e (e_to_e_out and e_out) ###########
    
    results = dbp.shoot_custom_query(one_triple_right % {'e': _uri})
    labels = ('e','e_to_e_out','e_out')

    #Insert results in subgraph
    insert_triple_in_subgraph(G, _results=results, 
                             _labels=labels, _direction=True, 
                             _origin_node=_uri, _filter_properties=True)
    
    ########### ?e ?p e (e_in and e_in_to_e) ###########
    
    results = dbp.shoot_custom_query(one_triple_left % {'e':_uri} )
    labels = ('e_in', 'e_in_to_e','e')
    
    #Insert results in subgraph
    insert_triple_in_subgraph(G, _results=results, 
                             _labels=labels, _direction=False, 
                             _origin_node=_uri, _filter_properties=True)
                        
    ########### e p eout . eout ?p ?e (e_out_to_e_out_out and e_out_out) ###########
    
    #Get all the eout nodes back from the subgraph.
    e_outs = []
    op = access.return_outnodes('e')
    for x in op: 
        for tup in x:
            e_outs.append(tup[1].getUri())
            
    labels = ('e_out','e_out_to_e_out_out','e_out_out')
    
    for e_out in e_outs:
        results = dbp.shoot_custom_query(one_triple_right % {'e' : e_out})
        
        #Insert results in subgraph
        insert_triple_in_subgraph(G, _results=results, 
                                 _labels=labels, _direction=True, 
                                 _origin_node=e_out, _filter_properties=True)
    
    ########### e p eout . ?e ?p eout  (e_out_in and e_out_in_to_e_out) ###########
    
    #Use the old e_outs variable
    labels = ('e_out_in','e_out_in_to_e_out','e_out')
    
    for e_out in e_outs:
        results = dbp.shoot_custom_query(one_triple_left % {'e' : e_out})
        
        #Insert results in subgraph
        insert_triple_in_subgraph(G, _results=results, 
                                 _labels=labels, _direction=False, 
                                 _origin_node=e_out, _filter_properties=True)
        
    ########### ?e ?p ein . ein p e  (e_in_in and e_in_in_to_e_in) ###########
    
    #Get all the ein nodes back from subgraph
    e_ins = []
    op = access.return_innodes('e')
    for x in op:
        for tup in x:
            e_ins.append(tup[0].getUri())
    
    
    labels = ('e_in_in','e_in_in_to_e_in','e_in')
    
    for e_in in e_ins:
        results = dbp.shoot_custom_query(one_triple_left % {'e': e_in})
        
        #Insert results in subgraph
        insert_triple_in_subgraph(G, _results=results, 
                                 _labels=labels, _direction=False, 
                                 _origin_node=e_in, _filter_properties=True)
        
    ########### ein ?p ?e . ein p e  (e_in_to_e_in_out and e_in_out) ###########
    
    #Use the old e_ins variable
    labels = ('e_in','e_in_to_e_in_out','e_in_out')
    
    for e_in in e_ins:
        results = dbp.shoot_custom_query(one_triple_right % {'e': e_in })
        
        #Insert results in subgraph
        insert_triple_in_subgraph(G, _results=results, 
                                 _labels=labels, _direction=True, 
                                 _origin_node=e_in, _filter_properties=True)
    
    
    #Pushed all the six kind of nodes in the subgraph. Done!
    return G

In [5]:
'''
    Testing the ability to create subgraph given a URI
'''
dbp =  db_interface.DBPedia(_verbose = True)
uri = 'http://dbpedia.org/resource/Bareilly'
graph = get_local_subgraph(uri)
f = open('output/graph.pickle','w+')
pickle.dump(graph,f)
subgraph.draw_graph(graph)



trying to print
completed printing


In [ ]:


def fill_specific_template(_template_id, _mapping):
	'''
		Using this function to map a string dict to a template.
		 -> create copy of template 
		 -> if literals, do something
		 -> push it in the list
		 -> periodic writes to disk (of the list)
	'''
	global sparql

	#TODO: Think if you wanna handle literals right here

	#Create copy of the template
	template = [x for x in templates if x['id'] == _template_id][0]
	template = copy.copy(template)
	template['query'] = template['template'] % mapping
	template['mapping'] = mapping

	print template
	raw_input()
	#Push it onto the SPARQL List
	#TODO: Add periodical writes
	sparql.append(template)

	return 0




def fill_templates(subgraph):
	templates = json.load(open('templates.py'))
	
	#TEMPLATE 1

	#For the first template, use ent1n and prop1n (their sequences are synced) to do this shizz
	for i in range(len(subgraph['ent1n'])):

		#Making the variables explicit (for the sake of readability)
		ent1n = subgraph['ent1n'][i]
		prop1n = subgraph['prop1n'][i] 

		#Create a mapping (in keeping with the templates' placeholder names)
		mapping = {'rel_to_a':prop1n,'ent_a':ent1n}

		#Throw it to a function who will put it in the list with appropriate bookkeeping
		fill_specific_template(_template_id = 1, _mapping = _mapping)

	# #TEMPLATE 2
	# for i in range
	


				



